# Preprocessing Data

## 2. Data Pre-Processing

### 2.a. Split into training and test datasets

In [ ]:
# pull from 1_raw_data and place in 2_proc_data

In [ ]:
# to create directories:
cwd = os.getcwd() #want this to be 2_proc_data

train_path = os.path.join(cwd,'Data','train_val')
test_path  = os.path.join(cwd,'Data','test')

make_dir(train_path)
make_dir(test_path)

In [ ]:
# Training set
train_files = ["historical", "ssp585", "ssp126", "ssp370","hist-aer","hist-GHG"] #pick a few CESM members?
X_train_xr, X_length  = prepare_predictor(train_files,train_path)
y_train_xr, y_length  = prepare_predictand(train_files,train_path)

# Test set
X_test_xr, _ = prepare_predictor('ssp245', data_path=test_path,time_reindex=False) #use remaining CESM members?
y_test_xr, _ = prepare_predictand('ssp245',data_path=test_path,time_reindex=False)

### 2.b. Organize all required variables into a dataframe 

In [ ]:
X_train_df = pd.DataFrame({"Chl": X_train_xr["CO2"].data,
                           "MLD": X_train_xr["MLD"].data,
                           "pCO2": X_train_xr["pC02"].data,
                           "SSS": X_train_xr["SSS"].data,
                           "SST": X_train_xr["SST"].data,
                          }, index=X_train_xr["pCO2"].coords['time'].data)

X_test_df  = pd.DataFrame({"Chl": X_test_xr["CO2"].data,
                           "MLD": X_test_xr["MLD"].data,
                           "pCO2": X_test_xr["pC02"].data,
                           "SSS": X_test_xr["SSS"].data,
                           "SST": X_test_xr["SST"].data,
                          }, index=X_test_xr["pCO2"].coords['time'].data)


y_train_df = y_train_xr["tas"].stack(dim=["latitude", "longitude"])
y_train_df = pd.DataFrame(y_train_df.to_pandas())

### 2.c. Data Normalization

In [ ]:
# Standardization
mean, std = X_train_df.mean(), X_train_df.std()

X_train_df   = (X_train_df - mean)/std
X_test_df    = (X_test_df - mean)/std

X_train = X_train_df.to_numpy()
y_train = y_train_df.to_numpy()
X_test = X_test_df.to_numpy()

print(X_train.shape,y_train.shape,X_test.shape)